In [5]:
import lightkurve as lk 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from pylab import *
import pandas as pd
import astropy.units as u
import math
import warnings
from IPython.display import Image
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', None)
import seaborn as sns
plt.rcParams['figure.dpi'] = 100
import os
from astroquery.simbad import Simbad

from multiprocessing import Pool
from astroquery.simbad import Simbad
#import read_mist_models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams["font.family"] = "Avenir"
from scipy import stats
import numpy as np

import pandas as pd
from pandas import DataFrame
pd.options.display.max_columns = 500
pd.options.display.max_rows = None

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import cm
%matplotlib inline
#import smplotlib

#import read_mist_models
#import mesa_reader as mr

import astropy.units as u
import astropy.coordinates as coord
from astropy.table import Table

from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
import astroquery.gaia
Vizier.ROW_LIMIT=-1
vsearch = Vizier(columns=["+_r","**"])#,column_filters={'Gmag': '<11'}
import astroquery.gaia
import glob, itertools

import logging
import warnings
from gyrointerp import gyro_age_posterior

from gyrointerp import get_summary_statistics

In [6]:
os.chdir('/Users/mlclark/Library/CloudStorage/GoogleDrive-mclark.astro@gmail.com/My Drive/UMa/final_uma')
rot_dir='rot_files'

In [ ]:
#replace with whatever files you have with rotation periods... for ex this one (ignoring that I already have ages in here)
rot_pers=pd.read_csv(os.path.join(rot_dir, '25_6_17_final_updated_pers.csv'))

In [11]:
def plot_colors(yval=-4.5, alpha=0.15, xmin=-0.5, xmax=3.5,fontsize=16):
    colors = [
        {"min": -3, "max": -0.26, "alpha": alpha, "color": '#5b7cff', "label": 'O'},
        {"min": -0.25, "max": -0.038, "alpha": alpha, "color": '#6988ff', "label": 'B'},
        {"min": -0.037, "max": 0.327, "alpha": alpha, "color": '#93aaff', "label": 'A'},
        {"min": 0.326, "max": 0.767, "alpha": alpha, "color": '#dddeff', "label": 'F'},
        {"min": 0.768, "max": 0.984, "alpha": alpha, "color": '#ffebdf', "label": 'G'},
        {"min": 0.983, "max": 1.85, "alpha": alpha, "color": '#ffb177', "label": 'K'},
        {"min": 1.85, "max": 7.0, "alpha": alpha, "color": '#ffa448', "label": 'M'}
    ]

    for c in colors:
        if c["min"] < xmax and c["max"] > xmin: # Check if the color is within xmin and xmax
            bprp1 = max(c["min"], xmin)
            bprp2 = min(c["max"], xmax)

            plt.axvspan(bprp1, bprp2, alpha=c["alpha"], color=c["color"], zorder=1)
            val = bprp1 + (bprp2 - bprp1) / 2
            plt.annotate(c["label"], xy=(val, yval), xytext=(val, yval), fontsize=fontsize,zorder=8)
    return

In [13]:
#if you don't have periods
def plot_Praesape():
    dp=pd.read_csv(r"G:\My Drive\downloads to transfer\douglas2019_praesape.csv",comment='#')

    dp=dp[dp.Prot>=0.2]
    dp['bp_rp_corr']=dp['BP-RP']-dp['E_BP-RP_']
    dp=dp.sort_values(by=['bp_rp_corr'])#(by=['bp_rp_corr'])
    dp=dp.reset_index()
    dp=dp[dp.RUWE<1.4]
    dp = dp.drop('index', axis=1)
    return dp
def median_difference(arr):
    median_value = np.median(arr)
    return np.nanmedian(np.abs(arr - median_value))

def find_prot(TIC, author, dataframe, minper,maxper, make_figs=True, edit_df=False):
    # Create the TIC ID
    TICID = f'TIC {TIC}'

    # Search for the light curve data
    lk_df = lk.search_lightcurve(TICID, author=author)
    
    if len(lk_df) == 0:
        raise ValueError(f"No light curves found for {TICID} by {author}")

    # Initialize lists to store results
    flux_err_medians = []
    sing_periods = []

    # Process each light curve
    for lc in lk_df:
        lc_sing = lc.download_all().stitch().remove_nans().remove_outliers(sigma=3)
        
        # Calculate periodogram and folded light curve once
        pg = lc_sing.to_periodogram(minimum_period=minper, maximum_period=maxper)
        fold = lc_sing.fold(pg.period_at_max_power)

        # Calculate flux error median and store the best period
        median_flux_err = np.nanmedian(fold.bin(0.1).flux_err)
        flux_err_medians.append(median_flux_err)
        sing_periods.append(pg.period_at_max_power.value)

    # Calculate the median of the best periods and their differences
    best_period = np.nanmedian(sing_periods)
    mmd_period = median_difference(np.array(sing_periods))

    # If make_figs is True, create the plots
    if make_figs:
        # Dynamically adjust figure size based on number of subplots (light curves)
        figure_height = max(10, (len(lk_df) + 1) * 4)  # Height should scale, but not be too small
        figure_width = 20  # Keep the width constant, but adjust based on your preference
        
        fig = plt.figure(figsize=(figure_width, figure_height), layout="constrained")
        gs = GridSpec(len(lk_df) + 1, 3, figure=fig)

        # Sort light curves by median flux error
        sorted_indices = np.argsort(flux_err_medians)
        
        # Define a dynamic font size based on the number of subplots
        base_fontsize = 20  # Base font size
        fontsize = base_fontsize / np.sqrt(len(lk_df))  # Scale the fontsize with sqrt of number of curves

        # Create subplots for each light curve, sorted by median flux error
        for plot_index in sorted_indices:
            lc_sing = lk_df[plot_index].download_all().stitch().remove_nans().remove_outliers(sigma=3)
            pg = lc_sing.to_periodogram(minimum_period=minper, maximum_period=maxper)
            fold = lc_sing.fold(pg.period_at_max_power)

            # Plot the periodogram
            axp = fig.add_subplot(gs[plot_index, 0])
            axp.plot(pg.period, pg.power, label=f'TIC{TIC}', color="gray")
            axp.text(5, pg.max_power, f'{pg.period_at_max_power}', fontsize=fontsize)
            axp.set_xlabel('Period [d]', fontsize=fontsize)
            axp.set_ylabel('Power', fontsize=fontsize)
            axp.set_title(f'Periodogram {lc_sing.sector} {author}', fontsize=fontsize)
            axp.legend(fontsize=fontsize)
            axp.tick_params(axis='both', labelsize=fontsize)

            # Plot the light curve with model
            lc_model_sing = pg.model(time=lc_sing.time, frequency=pg.frequency_at_max_power)
            axm = fig.add_subplot(gs[plot_index, 1])
            axm.plot(lc_sing.time.value, lc_sing.flux.value, color="gray", label=f'TIC{TIC}')
            axm.plot(lc_model_sing.time.value, lc_model_sing.flux.value, lw=5, ls='--', c='red', label=f'LS Model at period {pg.period_at_max_power}')
            axm.set_ylabel('Normalized Flux', fontsize=fontsize)
            axm.set_xlabel('Time - 2457000 [BTJD days]', fontsize=fontsize)
            axm.legend(fontsize=fontsize)
            axm.grid(alpha=0.2)
            axm.set_title(f'Sector {lc_sing.sector} {author}', fontsize=fontsize)
            axm.tick_params(axis='both', labelsize=fontsize)

            # Plot the folded light curve
            axf = fig.add_subplot(gs[plot_index, 2])
            fold.plot(label='Unbinned', ax=axf, color="gray")
            fold.bin(0.1).plot(label='Binned', ax=axf, lw=5, color="red")
            axf.set_ylabel('Normalized Flux', fontsize=fontsize)
            axf.set_xlabel('Time - 2457000 [BTJD days]', fontsize=fontsize)
            axf.legend(fontsize=fontsize)
            axf.grid(alpha=0.2)
            axf.set_title(f'Sector {lc_sing.sector} {author}', fontsize=fontsize)
            axf.text(0.5, 1.05, f'Median flux error= {np.nanmedian(fold.bin(0.1).flux_err):.2g}', transform=axf.transAxes, ha="center", va="bottom", fontsize=fontsize, color="blue")
            axf.tick_params(axis='both', labelsize=fontsize)

        # Additional plotting for Praesape (ensure dp2 and df2 are available in your code)
        dp2 = plot_Praesape()  # Ensure plot_Praesape() is defined elsewhere
        axr = fig.add_subplot(gs[-1, 0])
        axr.scatter(dp2['bp_rp_corr'], dp2['Prot'], label='Praesape Members: 670 Myr',
                    marker='P', color='blueviolet', zorder=1, edgecolors='black', linewidth=0.5, s=90)
        axr.scatter(dataframe[dataframe.TIC == TIC]['bp_rp_corr'], best_period, label=TICID,
                    marker='*', color='red', zorder=1, edgecolors='black', linewidth=0.5, s=500)
        axr.set_xlabel('BP-RP-E(BP-RP) (mag)', fontsize=fontsize)
        axr.set_ylabel('Rotation Period (days)', fontsize=fontsize)
        axr.set_xlim(0.1, 3.35)
        axr.set_ylim(-0.75, 44)
        plot_colors(41, 0.15)  # Ensure plot_colors is defined elsewhere
        axr.set_title('Rotation Period VS Color', fontsize=fontsize)
        axr.grid(alpha=0.2)

        axr.legend(fontsize=fontsize)
        fig.tight_layout()
        if edit_df: #to let you write periods in automatically. good for first time running, not as much if you're vetting periods individually, just overwrite manually when you're confident on a period
            dataframe.loc[dataframe[dataframe.TIC ==TIC].index, 'Median_period']=best_period
            dataframe.loc[dataframe[dataframe.TIC == TIC].index, 'Period_err']=mmd_period

        return best_period, mmd_period, sing_periods, len(lk_df), fig

    else:
        # If make_figs is False, just return the results without plotting
        return best_period, mmd_period, len(lk_df), None

In [ ]:
#if no teffs:
df4=pd.read_csv(os.path.join(rot_dir, 'EEM_dwarf_UBVIJHK_colors_Teff.txt'),skiprows=22,sep="\s+")
rot_pers['bp_rp_corr'] = pd.to_numeric(rot_pers['bp_rp_corr'], errors='coerce')  # Force non-numeric to NaN
df4['Bp-Rp'] = pd.to_numeric(df4['Bp-Rp'], errors='coerce')  # Force non-numeric to NaN

# Drop rows with NaN values in the relevant columns (optional, but it may help avoid issues)
#rot_pers = rot_pers.dropna(subset=['bp_rp_corr'])
df4 = df4.dropna(subset=['Bp-Rp', 'Teff'])

# Function to find the closest 'Bp-Rp' value in df4 to a given value in df3['bp_rp_corr']
def find_closest_teff(bp_rp_value):
    # Find the absolute difference between the bp_rp value and all values in df4['Bp-Rp']
    diff = (df4['Bp-Rp'] - bp_rp_value).abs()
    # Get the index of the minimum difference (closest value)
    closest_idx = diff.idxmin()
    # Return the corresponding 'teff' value from df4
    return df4.loc[closest_idx, 'Teff']

# Apply the function to your sources to create the new 'teff' column
rot_pers['Teff'] = rot_pers['bp_rp_corr'].apply(find_closest_teff)

In [10]:
age_cands=rot_pers[(rot_pers.Teff<6200)&(rot_pers.Teff>3800)&(rot_pers.ruwe<1.25)].reset_index(drop=True) #gyro interp bounds + ruwe cut as a binary flag

In [ ]:
age_grid = np.linspace(100, 4000, 500)  #how you want to compute the posteriors: this is 500 points between 100 Myr and 4 Gyr (approx every 8 Myr)
agepost=[]  #holds age posteriors
for i in range(len(age_cands)):
    Prot, Prot_err = age_cands['Median_Period'][i], np.maximum.reduce([0.2*age_cands['Median_Period'][i], age_cands['Period_err'][i]]) #period and error
    Teff, Teff_err = age_cands['Teff'][i], 150  #teff and error
    age_posterior = gyro_age_posterior(
        Prot, Teff,
        Prot_err=Prot_err, Teff_err=Teff_err,
        age_grid=age_grid
    )
    result = get_summary_statistics(age_grid, age_posterior) 
    agepost.append(age_posterior) #save the posterior for plotting or use in neural network etc!
    age_cands.loc[i, 'Median_rot_age']=result['median'] #save median age in csv
    age_cands.loc[i, 'Median_rot_up_err']=result['+1sigma'] #upper error
    age_cands.loc[i, 'Median_rot_low_err']=result['-1sigma'] #lower error
    age_cands.loc[i, 'peak_rot_age']=age_grid[np.where(age_posterior==np.max(age_posterior))[0][0]]  #also add the peak, deviates from median for asymmetric distributions, useful if you want to sort by peaks
agepost=np.array(agepost) #change list to array
#np.save('agepost_ruwe125.npy') save as a file to get again without having to rerun this cell (takes a while)